In [ ]:
# Imports

import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import folium
from folium.plugins import MarkerCluster
import warnings
from collections import Counter
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import HTML

# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
# Read data

# Read from disk
objects = pd.read_csv('../data/objects-all.csv')
vocabulary = pd.read_csv('../data/vocabulary-all.csv')
catalogs = pd.read_csv('../data/catalogs.csv', sep=';')
soltykoff = pd.read_csv('../data/sales-soltykoff.csv')

# Extract geographical places
geographical_places = vocabulary[vocabulary['type'] == 'origin']

# Format correctly strings and extract the sale year
objects['year'] = objects['catalog'].str.slice(0, 4).astype(pd.Int64Dtype())
geographical_places['name'] = geographical_places['name'].str.title()
objects['origin'] = objects['origin'].str.title()
objects['author'] = objects['author'].str.title()

# Change Period labels (caps)
def caps_periods(raw_period):
    if pd.isna(raw_period):
        return pd.NA
    arr = raw_period.split(', ')
    to_return = []
    for p in arr :
        if "e siècle" in p: to_return.append(p.replace('e siècle', '').upper() + 'e siècle')
        elif "louis " in p: to_return.append("Louis " + p.replace('louis ', '').upper())
        else: to_return.append(p.title())
    return ', '.join(to_return)

objects['period'] = [caps_periods(period) for period in objects['period']]

In [ ]:
# CSS for nbconvert 

HTML("""
<style>
    .jp-Cell { display: flex; justify-content: center; }     
    .jp-Cell-outputWrapper { width: 65%; display: flex; justify-content: center; }    
    .jp-Cell-inputWrapper { width: 65%; display: flex; justify-content: center; }     
</style>
""")

In [ ]:
# Fetch geographical places coordinates from Wikidata SPARQL endpoint

wikidata_ids = geographical_places['identifier'].dropna().tolist()

# SPARQL query with VALUES
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
query = f"""
    SELECT ?place ?lat ?lon
    WHERE {{
    VALUES ?place {{ {' '.join('wd:' + q for q in wikidata_ids)} }}
    ?place p:P625 ?statement.
    ?statement psv:P625 ?coords.
    ?coords wikibase:geoLatitude ?lat;
            wikibase:geoLongitude ?lon.
    }}
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Parse results into DataFrame
rows = []
for r in results["results"]["bindings"]:
    rows.append({
        "identifier": r["place"]["value"].split("/")[-1],
        "lat": float(r["lat"]["value"]),
        "lon": float(r["lon"]["value"])
    })

# Add the fectch coordinates
geographical_places = geographical_places.merge(pd.DataFrame(rows), on="identifier", how="left")

# Remove duplicates
geographical_places.drop_duplicates(subset=['name', 'identifier'], inplace=True)

# geographical_places

# Analysis 0: Cardinalities

In [ ]:
print(f"Total number of objects: {len(objects)}")
print()
print()
print('Attention, in following charts, records can have multiple origins, authors, period, and therefore can count multiple times.')
print()

## Origins ##

all_origins = ', '.join(objects['origin'].dropna().tolist()).split(', ')
origins_counts = pd.DataFrame.from_dict(Counter(all_origins), orient='index', columns=['count']).reset_index().sort_values('count', ascending=False)
fig = px.bar(origins_counts, x='index', y='count', title="Origins counts", labels={'index': 'Origin', 'count': 'Count'})
fig.show()
origin_unknown_nb = len(objects[pd.isna(objects['origin'])])
origin_unknown_percent = round(100 * (origin_unknown_nb / len(objects)), 2)
print(f"Number of lots with unknown origin: {origin_unknown_nb} ({origin_unknown_percent} %)")
print()
print()

## Authors ## 

all_authors = ', '.join(objects['author'].dropna().tolist()).split(', ')
all_authors = pd.DataFrame.from_dict(Counter(all_authors), orient='index', columns=['count']).reset_index().sort_values('count', ascending=False)
fig = px.bar(all_authors, x='index', y='count', title="Authors counts", labels={'index': 'Author', 'count': 'Count'})
fig.show()
author_unknown_nb = len(objects[pd.isna(objects['author'])])
author_unknown_percent = round(100 * (author_unknown_nb / len(objects)), 2)
print(f"Number of lots with unknown author: {author_unknown_nb} ({author_unknown_percent} %)")
print()
print()

## Periods ## 

all_periods = ', '.join(objects['period'].dropna().tolist()).split(', ')
all_periods = pd.DataFrame.from_dict(Counter(all_periods), orient='index', columns=['count']).reset_index().sort_values('count', ascending=False)
fig = px.bar(all_periods, x='index', y='count', title="Periods counts", labels={'index': 'Period', 'count': 'Count'})
fig.show()
period_unknown_nb = len(objects[pd.isna(objects['period'])])
period_unknown_percent = round(100 * (period_unknown_nb / len(objects)), 2)
print(f"Number of lots with unknown period: {period_unknown_nb} ({period_unknown_percent} %)")
print()
print()


## Unaligned vocabulary ##

# Object types
object_type_vocab = vocabulary[vocabulary['type'] == 'object_type']
unaligned_object_types = object_type_vocab[pd.isna(object_type_vocab['identifier'])]
unaligned_object_types_nb = len(unaligned_object_types)
unaligned_object_types_percent = round(100 * (unaligned_object_types_nb / len(object_type_vocab)), 2)
print(f'Total objects types number: {len(object_type_vocab)}')
print(f'Number of unaligned objects types: {unaligned_object_types_nb} ({unaligned_object_types_percent} %)')
print()

# Materials and techniques
mat_tech_vocab = vocabulary[vocabulary['type'] == 'material_technique']
unaligned_mat_techs = mat_tech_vocab[pd.isna(mat_tech_vocab['identifier'])]
unaligned_mat_techs_nb = len(unaligned_mat_techs)
unaligned_mat_techs_percent = round(100 * (unaligned_mat_techs_nb / len(mat_tech_vocab)), 2)
print(f'Total materials and/or technique number: {len(mat_tech_vocab)}')
print(f'Number of unaligned materials and/or technique: {unaligned_mat_techs_nb} ({unaligned_mat_techs_percent} %)')
print()

# Analysis 1: Object origins

In [ ]:
# Prepare origins

# Get only objects (lots) with origins
origins = objects[pd.notna(objects['origin'])]

number = len(origins)
percent = round(100 * (number / len(objects)), 2)
print(f'Number of objects with known origin: {number} ({percent} %)')

# Add the geocoordinates
origins = origins.merge(geographical_places, left_on='origin', right_on='name', how='left')

# Remove those without coordinates
origins = origins[pd.notna(origins['lat'])].copy()

# Create points labels
origins['label'] = origins['object_type'].fillna('Unknown') + ' - ' + origins['material_technique'].fillna('Unknown')

number = len(origins)
percent = round(100 * (number / len(objects)), 2)
print(f'Number of objects with known origin and known coordinates: {number} ({percent} %)')

### Map of all objects' origins across all catalogs

In [ ]:
# Display the map (all catalogs)

map = folium.Map(location=[20,0], zoom_start=3, tiles="OpenStreetMap")

# Add marker cluster
marker_cluster = MarkerCluster().add_to(map)

# Add points to cluster
for _, row in origins.iterrows():
    label = str(row['label'])
    folium.Marker(location=[row['lat'], row['lon']], tooltip=label, popup=label).add_to(marker_cluster)

map

### Map of all objects' origins for period 1839 to 1855

In [ ]:
# Display the map (1839 to 1855)

map = folium.Map(location=[20,0], zoom_start=3, tiles="OpenStreetMap")

# Add marker cluster
marker_cluster = MarkerCluster().add_to(map)

# Add points to cluster
origins_1839_1855 = origins[(origins['year'] >= 1839) & (origins['year'] <= 1855)]
for _, row in origins_1839_1855.iterrows():
    label = str(row['label'])
    folium.Marker(location=[row['lat'], row['lon']], tooltip=label, popup=label).add_to(marker_cluster)

map

### Map of all objects' origins for period 1861 to 1876

In [ ]:
# Display the map (1861 to 1876)

map = folium.Map(location=[20,0], zoom_start=3, tiles="OpenStreetMap")

# Add marker cluster
marker_cluster = MarkerCluster().add_to(map)

# Add points to cluster
origins_1861_1876 = origins[(origins['year'] >= 1861) & (origins['year'] <= 1876)]
for _, row in origins_1861_1876.iterrows():
    label = str(row['label'])
    folium.Marker(location=[row['lat'], row['lon']], tooltip=label, popup=label).add_to(marker_cluster)

map

### Map of all objects' origins for period 1880 to 1895

In [ ]:
# Display the map (1880 to 1895)

map = folium.Map(location=[20,0], zoom_start=3, tiles="OpenStreetMap")

# Add marker cluster
marker_cluster = MarkerCluster().add_to(map)

# Add points to cluster
origins_1880_1895 = origins[(origins['year'] >= 1880) & (origins['year'] <= 1895)]
for _, row in origins_1880_1895.iterrows():
    label = str(row['label'])
    folium.Marker(location=[row['lat'], row['lon']], tooltip=label, popup=label).add_to(marker_cluster)

map

# Analysis 2: Objects' origins and materials

In [ ]:
# Prepare objects (lots) with origins and material and origins

origins_materials = origins[pd.notna(origins['origin']) & pd.notna(origins['material_technique'])].copy().reset_index()

number = len(origins_materials)
percent = round(100 * (number / len(objects)), 2)
print(f'Number of objects with known origin and materials: {number} ({percent} %)')

In [ ]:
# Extract most commons materials and techniques to not overload chart

top_nb = 40

# Look for all unique mat and tech and count them
all_materials_techniques = ', '.join(origins_materials['material_technique']).split(', ')
counts = Counter(all_materials_techniques)

top = counts.most_common(top_nb)
top_strings = [item for item, _ in top]

### Map of most common objects' material/technique (top 40)

In [ ]:
# Create map
map = folium.Map(location=[20, 0], zoom_start=3)

# Add a layer for each material
top_strings.sort()
for mat in top_strings:
    fg = folium.FeatureGroup(name=mat,show=True)
    marker_cluster = MarkerCluster().add_to(fg)  
    selection = origins_materials[origins_materials['material_technique'].str.contains(mat)]
    for _, row in selection.iterrows():
        folium.Marker([row['lat'], row['lon']], tooltip=row['label'], popup=row['label']).add_to(marker_cluster)
    fg.add_to(map)

# Add layer control (checkboxes)
folium.LayerControl(collapsed=False).add_to(map)

map

# Analyse 3: Object types variations across periods

In [ ]:
# Prepare object types

object_types = objects[pd.notna(objects['object_type'])]

number = len(object_types)
percent = round(100 * (number / len(objects)), 2)
print(f'Number of objects with known object type: {number} ({percent} %)')

In [ ]:
# Extract most commons object type to not overload chart

top_nb = 20

# Get all unique object types and count them
all_object_types = ', '.join(object_types['object_type']).split(', ')
counts = Counter(all_object_types)

# Get most common object types
top = counts.most_common(top_nb)
top_strings = [item for item, _ in top]
top_strings.sort()

# To filter lots: because one lot can have multiple mat and techniques ("enamel, copper")
def is_in_top(object_type):
    for t in top_strings: 
        if t in object_type: return True
    return False
selection = object_types[[is_in_top(row['object_type']) for __annotations__, row in object_types.iterrows()]]

number = len(selection)
percent = round(100 * (number / len(objects)), 2)
print(f'Number of objects within the top {top_nb} object types: {number} ({percent} %)')


# Prepare the 3 periods
object_types_1839_1855 = Counter(', '.join(selection[(selection['year'] >= 1839) & (selection['year'] <= 1855)]['object_type'].tolist()).split(', '))
object_types_1861_1876 = Counter(', '.join(selection[(selection['year'] >= 1861) & (selection['year'] <= 1876)]['object_type'].tolist()).split(', '))
object_types_1880_1895 = Counter(', '.join(selection[(selection['year'] >= 1880) & (selection['year'] <= 1895)]['object_type'].tolist()).split(', '))

combined = {k: [object_types_1839_1855.get(k, 0), object_types_1861_1876.get(k, 0), object_types_1880_1895.get(k, 0)] for k in top_strings}

# Periods Labels
x = ['1839 to 1855', '1861 to 1876', '1880 to 1895']

# Create figure
fig = go.Figure()

# Add one chart for each object types
for key, counts in combined.items():
    fig.add_trace(go.Scatter(x=x, y=counts, mode='markers+lines', name=key))

# Create the chart
fig.update_layout(
    title=f"Evolution of object types ({top_nb} most common) presence in auctions across periods",
    xaxis_title="Periods",
    yaxis_title="Counts",
    template="plotly_white"
)

fig.show()

# Analyse 4: Production periods variation across periods

In [ ]:
# Prepare periods

periods = objects[pd.notna(objects['period'])]

number = len(periods)
percent = round(100 * (number / len(objects)), 2)
print(f'Number of objects with known production periods: {number} ({percent} %)')

In [ ]:
# Extract most commons periods to not overload chart

top_nb = 10

# Get all unique periods and count them
all_periods = ', '.join(periods['period']).split(', ')
counts = Counter(all_periods)

# Get most common periods
top = counts.most_common(top_nb)
top_strings = [item for item, _ in top]
top_strings.sort()

# To filter lots: because one lot can have multiple periods ("louis xvi, renaissance")
def is_in_top(period):
    for t in top_strings: 
        if t in period: return True
    return False
selection = periods[[is_in_top(row['period']) for __annotations__, row in periods.iterrows()]]

number = len(selection)
percent = round(100 * (number / len(objects)), 2)
print(f'Number of objects within the top {top_nb} periods: {number} ({percent} %)')

# Prepare the 3 periods
periods_1839_1855 = Counter(', '.join(selection[(selection['year'] >= 1839) & (selection['year'] <= 1855)]['period'].tolist()).split(', '))
periods_1861_1876 = Counter(', '.join(selection[(selection['year'] >= 1861) & (selection['year'] <= 1876)]['period'].tolist()).split(', '))
periods_1880_1895 = Counter(', '.join(selection[(selection['year'] >= 1880) & (selection['year'] <= 1895)]['period'].tolist()).split(', '))

combined = {k: [periods_1839_1855.get(k, 0), periods_1861_1876.get(k, 0), periods_1880_1895.get(k, 0)] for k in top_strings}

# Periods Labels
x = ['1839 to 1855', '1861 to 1876', '1880 to 1895']

# Create figure
fig = go.Figure()

# Add one chart for each periods
for key, counts in combined.items():
    fig.add_trace(go.Scatter(x=x, y=counts, mode='markers+lines', name=key))

# Create the chart
fig.update_layout(
    title=f"Evolution of production periods ({top_nb} most common) presence in auctions across periods",
    xaxis_title="Periods",
    yaxis_title="Counts",
    template="plotly_white"
)

fig.show()

# Analyse 5: Auctioneers and experts

In [ ]:
# Get all auctioneers
auctioneer = '\n'.join(catalogs['auctioneer_names'].dropna().tolist()).split('\n')
auctioneer_counts = Counter(auctioneer)

# Sort items by count descending
sorted_items = auctioneer_counts.most_common()
labels, counts = zip(*sorted_items)

# Create a bar chart
fig = go.Figure([go.Bar(x=labels, y=counts)])

# Optional: add title and axis labels
fig.update_layout(
    title='Auctioneer participations in auction across all catalogues',
    xaxis_title='Items',
    yaxis_title='Count'
)

fig.show()

In [ ]:
# Get all expert names
experts = '\n'.join(catalogs['experts_names'].dropna().tolist()).split('\n')
experts = [x for x in experts if x != '']
experts_counts = Counter(experts)

# Sort items by count descending
sorted_items = experts_counts.most_common()  # returns list of tuples (item, count)
labels, counts = zip(*sorted_items)  # unzip into separate lists

# Create a bar chart
fig = go.Figure([go.Bar(x=labels, y=counts)])

# Optional: add title and axis labels
fig.update_layout(
    title='Experts participations in auction across all catalogues',
    xaxis_title='Items',
    yaxis_title='Count'
)

fig.show()

# Analyse 6: Soltykoff sales

In [ ]:
# Replace all "id" (idem) by the one above
for i, row in soltykoff.iterrows():
    if row['buyer'] == 'id': 
        soltykoff.at[i, 'buyer'] = soltykoff.iloc[i-1]['buyer']

# Get the sum of all buyers
buyers = soltykoff.groupby('buyer').sum().reset_index()[['buyer', 'price']].sort_values('price', ascending=False)

# Calculate total and threshold
total = buyers['price'].sum()
threshold = 0.03  # In percent

# Mark small buyers as 'Others'
buyers['buyer'] = buyers.apply(lambda row: row['buyer'] if row['price']/total >= threshold else f'Others (lower than {threshold*100} %)', axis=1)

# Re-aggregate after grouping small buyers
buyers = buyers.groupby('buyer', as_index=False)['price'].sum()

# Plot interactive pie chart
fig = px.pie(buyers, names='buyer', values='price', title='Total Price by Buyer (in Francs)', color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_traces(textinfo='label+value+percent')
fig.update_layout(height=600, showlegend=False)
fig.show()

In [ ]:
# Get, thanks to the index the lots bought
soltykoff_objects = objects[objects['catalog'] == '1861-04-08_Prince-Soltykoff']
soltykoff_ = soltykoff.merge(soltykoff_objects[['index', 'object_type']], on='index', how='inner')

In [ ]:
fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]], subplot_titles=["Seillère buyings", "Webb buyings", "Roussel buyings"])

## SEILLERE ##

seillere = soltykoff_[soltykoff_['buyer'] == 'Seillère']
total = seillere['price'].sum()
threshold = 0.03  # 5% of total
seillere['object_type'] = seillere.apply(lambda row: row['object_type'] if row['price']/total >= threshold else f'other (lower than {threshold * 100} %)', axis=1)
seillere = seillere.groupby('object_type', as_index=False)['price'].sum().sort_values('price')
seillere_pie = px.pie(seillere, names='object_type', values='price', title='Seillière buyings (in Franc)', color_discrete_sequence=px.colors.qualitative.Pastel)
fig.add_trace(go.Pie(labels=seillere['object_type'], values=seillere['price'], name="Seillière buyings (in Franc)", textinfo="label+value"), 1, 1)

## WEBB ##

webb = soltykoff_[soltykoff_['buyer'] == 'Webb']
total = webb['price'].sum()
threshold = 0.03  # 5% of total
webb['object_type'] = webb.apply(lambda row: row['object_type'] if row['price']/total >= threshold else f'other (lower than {threshold * 100} %)', axis=1)
webb = webb.groupby('object_type', as_index=False)['price'].sum().sort_values('price')
webb_pie = px.pie(webb, names='object_type', values='price', title='Seillière buyings (in Franc)', color_discrete_sequence=px.colors.qualitative.Pastel)
fig.add_trace(go.Pie(labels=webb['object_type'], values=webb['price'], name="Seillière buyings (in Franc)", textinfo="label+value"), 1, 2)

## ROUSSEL ##

roussel = soltykoff_[soltykoff_['buyer'] == 'Roussel']
total = roussel['price'].sum()
threshold = 0.03  # 5% of total
roussel['object_type'] = roussel.apply(lambda row: row['object_type'] if row['price']/total >= threshold else f'other (lower than {threshold * 100} %)', axis=1)
roussel = roussel.groupby('object_type', as_index=False)['price'].sum().sort_values('price')
roussel_pie = px.pie(roussel, names='object_type', values='price', title='Seillière buyings (in Franc)', color_discrete_sequence=px.colors.qualitative.Pastel)
fig.add_trace(go.Pie(labels=roussel['object_type'], values=roussel['price'], name="Seillière buyings (in Franc)", textinfo="label+value"), 1, 3)


fig.update_layout(
    showlegend=False, 
    title=dict(text="Details about the 3 higher buyers of Soltykoff auction sale", x=0.5, y=0.95)
)
fig.show()